# YouTube 댓글 크롤링

In [52]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import sys
import urllib
import time
import pandas as pd
import re
from pandas import DataFrame, Series
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys


# 특정 키워드에 대한 영상들의 url을 저장하는 함수
def video_url_crawling():

        data = []
        keyword = input('Keyword : ')

        driver = wd.Chrome('./chromedriver.exe')
        driver.maximize_window()

        YOUTUBE_URL = f'https://www.youtube.com/results?search_query={keyword}&sp=CAM%253D'
        driver.get(YOUTUBE_URL)


        print('The scrolling starts moving to the bottom of the main page.')
        body = driver.find_element_by_tag_name("body")
        

        while True:
            body.send_keys(Keys.PAGE_DOWN) # 스크롤 다운
            time.sleep(0.4)
            
            items=driver.find_elements_by_css_selector('#message') # id = '#message' 에 해당하는 값을 가져옴
            if items[0].text == "결과가 더 이상 없습니다.": # scroll을 맨 끝까지 내렸을 때 message의 내용
                break   # 반복문 탈출
            
        print('Arrived at the end of the main page')
        print('Start to get the url of the video')
        
        # 동영상 제목, url을 가지고 있는 class를 가져옴.
        items = driver.find_elements_by_css_selector('#video-title')

        for idx in items:
            if(idx.get_attribute('href') is not None):
                # 한글 깨짐 방지
                text = idx.text

                for i in range(len(text)):
                    if not ((0 <= ord(text[i]) < 128) or (0xac00 <= ord(text[i]) <= 0xd7af)):
                        text = text.replace(text[i], ' ')
                data.append([text, idx.get_attribute('href')])

        driver.close()

        print('Finish previous working')
        print('The data is being written to the csv file.')
        
        # csv 파일에 저장 [동영상 제목, 동영상 url]
        dataframe = pd.DataFrame(data, columns=["title", "url"])
        dataframe.to_csv('./youtube_url_collection.csv', mode = 'a', encoding='cp949')
        
        print('Finish working')        
        
        
# csv파일에 저장된 영상 url를 이용해 댓글을 크롤링하는 함수
def video_comment_crawling():
    data = []
    df = pd.read_csv('./youtube_url_collection.csv', encoding = 'cp949')

    driver = wd.Chrome('./chromedriver.exe')
    driver.maximize_window()

    temporary_storage_num = 1
    for i in range(len(df.index)):
        title = df['title'][i]
        link = df['url'][i]

        print(f'Start comment crawling : title = {title}')
              
        driver.get(link)
        time.sleep(2)

        count = 0
        body = driver.find_element_by_tag_name("body")

        # print('The scrolling starts moving to the bottom of the comment page.')
        
        # 댓글 데이터를 가져옴
        last = driver.find_elements_by_css_selector('#content-text')

        while True:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.4)
            new = driver.find_elements_by_css_selector('#content-text')

            if new == last:
                if count == 10:
                    break
                count += 1

            else:
                count = 0

            last = new
            
        # print('Arrived at the end of the comment page')
        
        for idx in new:
            # 한글 깨짐 방지
            text = idx.text

            for idx in range(len(text)):
                if not ((0 <= ord(text[idx]) < 128) or (0xac00 <= ord(text[idx]) <= 0xd7af)):
                    text = text.replace(text[idx], ' ')

            data.append([title, text])
            
            
        if temporary_storage_num == 1:
            dataframe = pd.DataFrame(data, columns=["title", "content"])
            dataframe.to_csv('./youtube_comment.csv', mode = 'a', encoding='cp949')
            data = []
            
        if temporary_storage_num % 3 == 0:
            dataframe = pd.DataFrame(data, columns=["title", "content"])
            dataframe.to_csv('./youtube_comment.csv', mode = 'a', encoding='cp949', header=False)
            data = []
            
        temporary_storage_num += 1

    driver.close()
    print('Finish comment crawling')
    
    # 댓글 데이터를 csv 파일에 저장
    dataframe = pd.DataFrame(data, columns=["title", "content"])
    dataframe.to_csv('./youtube_comment.csv', mode = 'a', encoding='cp949')

    print('Finish working')

In [ ]:
video_url_crawling() # 키워드에 대한 영상들의 url 저장

In [ ]:
video_comment_crawling() # url에 접속하여 댓글들 가져옴

Start comment crawling : title = 챗봇, 개념만 이해하기
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 코딩없이 챗봇 제작하기 풀영상 / 챗봇 기초부터 제작까지
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 안녕...  AI 챗봇 '이루다' 서비스 중단, 이전에도 똑같은 일 있었다 / 스브스뉴스
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 인공지능 대화처리(챗봇) 실습 사이트 3곳
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 챗봇(chat bot)이 뭐죠? [IT알려줌-이슈]
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = Machine Learning for Kids (10) 챗봇 1/3
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Star

The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 중앙대학교 챗봇(CHARLI) 홍보 동영상
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 파이썬 장고(Django)로 챗봇 만들기 - Part 1. Python Chatbot 기본
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 1시간만에 API이용 챗봇 만들기! - www.playchat.ai
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 인공지능(AI) 스피커 챗봇의 원리!!!
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = AI챗봇 이루다 차단하고 삭제까지 하셨다구요?? 다시 대화하는법 알려드립니다
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 09 질문에 답변할 

The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 003. 스크래치 - 챗봇
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = part 06 금융 관리 챗봇 (주식 분석)
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 초간단 챗봇구축 네이버 톡톡 채팅서비스;
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 스낵면이 끓기 전에 만들 수 있는 FAQ챗봇
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 병원상담 AI 인공지능 챗봇 의사소통 카카오톡 버전
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = [디잼] 카카오톡 챗봇 텍스트형 시현 영상
The scrolling starts moving to the bottom of the 

Arrived at the end of the comment page
Start comment crawling : title = Machine Learning for Kids (11) 챗봇 2/3
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = [챗봇 사례] 챗봇, 어디다 쓰나? 챗봇 강좌 챗봇 원리 챗봇 만들기
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = [2강] 챗봇, 인간의 친구가 되다
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 13-1. AI와 챗봇으로 IBM과 맞장뜨다, 와이즈넛의 성공 비결은?(강용성 대표, 와이즈넛)
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = [YAP(얍) NEWS] 스마트오더 기술도 이제는 챗봇이 대세!
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 구글 다이얼로그 플로우(Dialog Flow)로 무료 챗봇 만들기
The scrolling starts moving t

Arrived at the end of the comment page
Start comment crawling : title = 사람의 말을 이해하는 인공지능 챗봇! - '머니브레인' 장세영 대표 / YTN 사이언스
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 금융권 디지털 자문 시장 전망과 비대면 모바일 챗봇 확산 가능성
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = [Office 365 꿀팁] 439. 이제 챗봇도 직접 만들자! Power Virtual Agent
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 개발자 없이 쉽게 챗봇을 만드는 방법!
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = part 07 법률 용어 / 법원 상담 챗봇
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = SCHOOOL Lingobot: 영어 스피킹 연습 챗봇
The scrolling starts moving to the

The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 스마트 지능형 챗봇
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = "이루다 챗봇 '오염된 데이터' 러닝한 것"
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 구글 설문 챗봇 테스트  fobi.io
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = [스프링살롱] 타로챗봇 라마마를 만든 띵스플로우 이수지 대표 #1
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = [AI챗봇 이루다] 자기자신과의 대화
The scrolling starts moving to the bottom of the comment page.
Arrived at the end of the comment page
Start comment crawling : title = 챗봇마스터
The scrolling starts moving to the bottom of the comment page.
Arriv